In [1]:
import os
from os.path import join, getmtime
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
adate=datetime.now().strftime('%Y-%m-%d')
adate="2025-01-21"
dateList=[
    "2025-01-22","2025-01-21","2025-01-20",
    "2025-01-17","2025-01-16","2025-01-15","2025-01-14","2025-01-13",
    ]

In [5]:
new_DailyResult_path_List=[]
new_RunRealTimeStock_path_List = [f'D:\\Project\\Jupyter\\Stock\\Main\\Result\\DailyRunRealTimeStock\\{adate}.xlsx']
new_DailyResult_path_List = [ f"D:\Project\Jupyter\Stock\Main\Result\DailyFiles\\{date}.xlsx" for date in dateList] 
data=StockBaseData=pd.read_csv('BaseInfo/StockBaseData.csv', converters={'公司代號': str})

_df =  pd.DataFrame()
for path in new_RunRealTimeStock_path_List:
    try:
        _df=pd.concat([pd.read_excel(path), _df])
    except:
        print('error----->'+path)
    
_df2 = pd.DataFrame()
for path in new_DailyResult_path_List:
    try:
        _df2=pd.concat([pd.read_excel(path), _df2])
    except:
        print('error----->'+path)
        
df=pd.merge( _df2,_df[pd.notna(_df['now_price'])], left_on='stock_number', right_on='stock_number', how='outer')

#print(_df2.dtypes)
#print(_df.dtypes)

error----->D:\Project\Jupyter\Stock\Main\Result\DailyFiles\2025-01-22.xlsx


In [6]:
df.rename(columns={'日期_x': '日期'}, inplace=True)
df.rename(columns={'now_price_y': 'now_price'}, inplace=True)
df.rename(columns={'change_price_y': 'change_price'}, inplace=True)
df.rename(columns={'change_quote_y': 'change_quote'}, inplace=True)
df['日期'] = df['日期'].dt.strftime('%Y-%m-%d')

In [7]:
def  GetStockInfoByID(StocKID,strtype):
    try :
        result=data[data['公司代號']==StocKID]
        if strtype=='name':
            return result.iloc[0]['公司簡稱']
        if strtype=='category':
            if result.iloc[0]['Type']=="上櫃" :
                return result.iloc[0]['上櫃公司產業類別']
            else:
                return result.iloc[0]['上市公司產業類別']
        if strtype=='type':
            return result.iloc[0]['Type']
    except :
        print(StocKID)

In [8]:
def stock_recommend(row):
    _str=''
    _quote=float( row['_quote'])
    _level=float( row['level'])

    if int(row['遠交易量'])>0 and _quote > -2:   
        _str+='遠期量推'
    if row['短增'] and int(row['短交易量'])>0 and int(row['遠交易量'])>0 and _quote > -1:   
        _str+='近期量推'
        if int(row['成交股數'])>50000:
            _str+='量大'
   
        
    if row['interval_type']=='負區間':
        if  _level >= 40:
            _str+='探底'
            if row['Signal_Balance_uptrend_3days'] and  row['Signal_Balance_uptrend']  :
                _str+='有撐'  
            else:
                _str=''
    if row['短增'] and int(row['短交易量'])>0 and int(row['遠交易量'])>0 and _quote > 4:
        _str+='ShowTime'
    return _str
def stock_recommend2(row):
    _str=''
    _quote=float( row['_quote'])
    _level=float( row['level'])
    if len(row['recommend'] )>0:
        
        if row['interval_type']=='正區間':
            if  _level >= 90:
                _str+='追高' 

        if row['interval_type']=='負區間':
            if  _level >= 40:
                _str+='探底'       

            if row['Signal_Balance_uptrend_3days'] and  row['Signal_Balance_uptrend']  :
                _str+='有撐'  
            
    return _str

def stock_LevelArea(row):
    _level=float( row['level'])

    if row['interval_type']=='正區間': 
        _level=_level*1

    if row['interval_type']=='負區間':
        _level=_level*-1
            
    return _level

def  stock_quote(row):
      return str(format(float(row['漲跌價差'])/float(row['開盤價'])*100, '.2f'))    
    
def  diff_quote(row):
    try :
        if float(row['收盤價'])!=0:
           # return str(format((float(row['change_quote'])-float(row['_quote'])) /float(row['_quote'])*100, '.2f'))    
            return format((float(row['now_price'])-float(row['收盤價'])) /float(row['收盤價'])*100, '.2f')

    except FileNotFoundError:
        print(row['_quote'])

def trading_volume(row):
    data=row['成交股數']
    try :
         if row['Type2']=='上市': 
                data=data/1000
    except FileNotFoundError:
          print(data)
        
    return data

In [9]:
# Apply stock information based on 'stock_number' and add new columns
df['Type0'] = df.apply(lambda x: f"{GetStockInfoByID(str(x['stock_number']),'name')}", axis=1)
df['Type1'] = df.apply(lambda x: f"{GetStockInfoByID(str(x['stock_number']),'category')}", axis=1)
df['Type2'] = df.apply(lambda x: f"{GetStockInfoByID(str(x['stock_number']),'type')}", axis=1)
df['Type'] = df['Type0'] + '-' + df['Type1'] + '-' + df['Type2']

# Clean and convert 'change_quote' to numeric, handle invalid values
df['change_quote'] = df['change_quote'].astype(str).str.replace("+", "", regex=False).str.replace("%", "", regex=False)
df['change_quote'] = pd.to_numeric(df['change_quote'], errors='coerce').fillna('0.00').astype(str)

# Convert 'now_price', '收盤價' to numeric with '0.00' fallback
df['now_price'] = pd.to_numeric(df['now_price'], errors='coerce').fillna('0.00').astype(str)
df['收盤價'] = pd.to_numeric(df['收盤價'], errors='coerce').fillna('0.00').astype(str)

# Clean up other columns and fill missing values
df['Signal_Balance_roll'] = pd.to_numeric(df['Signal_Balance_roll'], errors='coerce').fillna(0)
df['MA_break'] = df['MA_break'].fillna(False)
df['MACD_golden_cross'] = df['MACD_golden_cross'].fillna(False)
df['stock_number'] = df['stock_number'].astype(str)

# Handle 'Trend' column, filling NaN with '--' and replacing '0' with '-'
df['Trend'] = df['Trend'].fillna("--").astype(str).replace("0", "-")

# Convert signal columns to string and perform calculations
df['Signal_Balance_uptrend_3days'] = df['Signal_Balance_uptrend_3days'].astype(str)
df['Weighted_Signa_over_threshold'] = df['Weighted_Signa_over_threshold'].astype(str)
df['Signal_Balance_roll'] = (df['Signal_Balance_roll'] * 10).astype(str)

# Clean and convert 'Volume_Oscillator'
df['Volume_Oscillator'] = df['Volume_Oscillator'].fillna('0').astype(int).astype(str)

# Multiply 'level' and convert 'now_price' to string
df['level'] = (df['level'] * 100).astype(str)
df['now_price'] = df['now_price'].astype(str)

# Calculate and clean up trading volume metrics
df['短交易量'] = (df['成交金額'] - df['Volume_MA_short']) / df['Volume_MA_short']
df['遠交易量'] = (df['Volume_MA_short'] - df['Volume_MA_long']) / df['Volume_MA_long']
df['短增'] = df['短交易量'] > df['遠交易量']

# Calculate price difference and volume-price ratio
df['漲跌價差'] = pd.to_numeric(df['漲跌價差'], errors='coerce').fillna(0)
df['量增與漲幅倍率'] = (df['漲跌價差'] / df['開盤價']) * 100 / df['短交易量']

# Flag rows with significant volume increase and positive price change
df['量增'] = (df['短交易量'] > df['遠交易量']) & (df['短交易量'] > 0) & (df['遠交易量'] > 0) & (df['量增與漲幅倍率'] > 0)

# Multiply trading volumes by 100 and fill NaN with '0'
df['短交易量'] = (df['短交易量'] * 100).fillna('0').astype(int)
df['遠交易量'] = (df['遠交易量'] * 100).fillna('0').astype(int)

# Calculate short-to-long volume ratio, handle infinite values
df['短遠交易比'] = (df['短交易量'] / df['遠交易量']).replace([np.inf, -np.inf], 0).fillna(0).astype(int)

# Convert trading volumes to strings
df['短遠交易比'] = df['短遠交易比'].astype(str)
df['短交易量'] = df['短交易量'].astype(str)
df['遠交易量'] = df['遠交易量'].astype(str)

# Clean up 'Volume_Price_Change_sum' and '量增與漲幅倍率'
df['Volume_Price_Change_sum'] = df['Volume_Price_Change_sum'].fillna('0').astype(str)
df['量增與漲幅倍率'] = df['量增與漲幅倍率'].fillna('0').astype(str)

# Apply custom functions for additional calculations
df['_quote'] = df.apply(stock_quote, axis=1)
df['recommend'] = df.apply(stock_recommend, axis=1)
df['recommend2'] = df.apply(stock_recommend2, axis=1)
df['LevelArea'] = df.apply(stock_LevelArea, axis=1)
df['diff_quote'] = df.apply(diff_quote, axis=1)
df['成交股數'] = df.apply(trading_volume, axis=1)

# Replace all '-' values with 0
df.replace('-', 0, inplace=True)

In [10]:
def jug (row,logic_type,predict):
  
    _str='N'
    if logic_type=='多頭排列':
        
        # 【邏輯一】
        if( float(row['EMA_12'])>float(row['EMA_26']) 
           and  float(row['now_price']) >float(row['EMA_12']) 
           and  float(row['Volume_Price_Change_sum']) >0
           and  float(row['遠交易量']) >0
           and  float( row['change_quote'])>-7.5
           and  (float( row['change_quote'])>-3 and  float( row['成交股數'])>5000)
           and   row['recommend'] !=''
           and   row['Is_Entangled']   # 當短期移動平均線的波動率大於長期移動平均線的波動率時，標記為糾纏狀態
           #and   row['Close_above_MA_5days']  # 計算連續 5 日收盤價都高於短期移動平均線的天數
          ) :
            _str='T'
            
        if( float(row['成交股數']) <100
           and row['interval_type'] =='負區間'
           and  float(row['level'] )>50
           and  float(row['Volume_Price_Change_sum']) <0
          ):
            _str='F'
            
        if( float(row['遠交易量']) <-20
           and  float(row['遠交易量'])<-20
           and  float(row['Volume_Price_Change_sum']) <-500
          ):
            _str='F'
        if( float(row['EMA_12'])<float(row['EMA_26']) ):
             _str='F'
                        
    elif logic_type=='':
        # 【邏輯二】
        if row['']>1:
            _str='T'
        else:
            _str='F'

    if predict!=True:
        #事實判定
        _quote=float( row['change_quote'])
        if _quote>0:
            _str+='T'
        else:
            _str+='F'
       
    return _str

In [11]:
aa=df
aa['預測']  =df.apply(lambda x: jug(x,'多頭排列',True), axis=1)
aa['判定']  =df.apply(lambda x: jug(x,'多頭排列',False), axis=1)
pd.pivot_table(data=aa, index='判定', aggfunc=pd.Series.nunique).sort_values(by=['判定'], ascending=False)

,%D,%K,Analysis,Bar_Color,Bollinger_MA,Bollinger_STD,Bollinger_breakout,Bollinger_lower,Bollinger_upper,Close_above_MA,Close_above_MA_5days,DIF,Date,Date_,EMA_12,EMA_26,Highest_High,Is_Entangled,KD_golden_cross,LevelArea,Lowest_Low,MACD,MACD-SL,MACD_golden_cross,MA_break,MA_long,MA_long_change,MA_long_volatility,MA_short,MA_short_change,MA_short_volatility,Price_Change,RSI,RSI_rebound,Signal_Balance,Signal_Balance_roll,Signal_Balance_roll_diff,Signal_Balance_uptrend,Signal_Balance_uptrend_3days,Signal_Line,Trend,Type,Type0,Type1,Type2,Volume_Change,Volume_Change_sum,Volume_MA_long,Volume_MA_short,Volume_Oscillator,Volume_Price_Change,Volume_Price_Change_sum,Volume_breakout,Weighted_Signa_over_threshold,Weighted_Signal,Weighted_Signal_golden_cross,Weighted_Signal_roll,Weighted_Signal_roll_,Year,_quote,change_price,change_price_x,change_quote,change_quote_x,diff_quote,interval_type,level,lower_bound,now_price,now_price_x,recommend,recommend2,stock_number,upper_bound,年月日,成交筆數,成交股數,成交金額,收盤價,日期,日期_y,最低價,最高價,漲跌價差,短交易量,短增,短遠交易比,遠交易量,量增,量增與漲幅倍率,開盤價,預測
判定,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TT,32,15,9,2,39,39,2,39,39,2,2,39,7,7,39,39,30,1,2,7,25,39,39,2,2,39,37,39,38,35,39,32,38,1,3,9,7,2,2,39,3,19,19,11,2,39,39,39,39,30,39,39,2,2,8,2,23,26,1,34,18,33,18,36,28,2,6,19,19,34,6,3,19,19,7,39,39,39,34,7,1,37,37,31,38,2,17,30,2,38,38,1
TF,39,21,11,2,48,48,2,48,48,2,2,48,7,7,48,48,38,1,2,8,29,48,48,2,2,48,45,48,48,48,48,42,48,2,3,15,12,2,2,48,4,23,23,10,2,48,48,48,48,44,48,48,2,2,8,2,34,37,1,47,17,43,23,47,39,2,7,23,23,45,8,3,23,23,7,48,48,48,45,7,1,47,47,39,45,2,15,44,2,48,46,1
NT,765,480,53,2,908,920,2,920,920,2,2,978,9,9,974,974,418,2,2,11,373,971,978,2,2,908,581,891,889,567,930,265,855,2,4,17,19,2,3,978,5,222,222,29,2,897,927,961,974,191,977,975,2,3,10,2,167,162,1,540,95,233,159,523,531,2,8,221,211,723,10,3,222,221,7,751,847,974,724,7,1,716,708,184,278,2,86,191,2,897,714,1
NF,1220,779,58,2,1344,1386,2,1388,1387,2,2,1479,8,8,1467,1478,526,2,2,14,523,1477,1479,2,2,1344,761,1348,1331,743,1424,335,1274,2,4,20,19,2,3,1479,5,306,306,31,2,1300,1340,1436,1454,202,1464,1464,2,3,10,2,194,173,1,712,75,282,170,678,694,2,8,306,284,946,13,4,306,306,7,1071,1214,1457,947,7,1,953,936,227,309,2,85,202,2,1348,962,1
FT,3237,1792,63,2,3646,3850,2,3857,3856,2,2,4129,7,7,4115,4115,1224,2,2,14,719,4115,4129,2,2,3646,1224,3858,3242,1135,3958,495,3290,2,3,11,15,2,2,4129,4,635,635,34,2,3048,3217,4090,4102,184,4028,4063,2,2,9,2,166,164,1,939,138,361,248,879,1025,2,7,635,523,1552,13,4,635,635,7,1410,2841,3931,1553,7,1,1562,1592,277,367,2,140,184,2,3721,1581,1
FF,4866,2548,69,2,5217,5733,2,5739,5740,2,2,6396,7,7,6300,6304,1407,2,2,14,970,6304,6396,2,2,5217,1475,5744,4297,1302,5905,527,4610,2,4,12,17,2,2,6396,4,993,993,35,2,4675,4870,6329,6364,201,6128,6217,2,2,10,2,198,197,1,1080,103,372,268,1019,1106,2,7,969,734,1751,13,4,993,969,7,1880,4351,6012,1752,7,1,1745,1767,292,391,2,152,201,2,5691,1746,1


In [12]:
aa['Volume_Price_Change_sum'] = pd.to_numeric(aa['Volume_Price_Change_sum'], errors='coerce')
aa['遠交易量'] = pd.to_numeric(aa['遠交易量'], errors='coerce')
aa['短交易量'] = pd.to_numeric(aa['短交易量'], errors='coerce')
aa['_quote'] = pd.to_numeric(aa['_quote'], errors='coerce')
aa['Highest_High'] = pd.to_numeric(aa['Highest_High'], errors='coerce')
aa['now_price'] = pd.to_numeric(aa['now_price'], errors='coerce')
aa['收盤價'] = pd.to_numeric(aa['收盤價'], errors='coerce')
aa['過壓力區']= aa['收盤價']/aa['Lowest_Low'] 


In [13]:
aa['TTYPE']=''
aa['TJug']=''

# 低量，若過壓力區越大==> 有撐  ( 可以找過壓力區>1.0 ，1的話代表當前價位幾乎等於近期價位)，可再回來看看
x_type='TYPE(--)'
aa['TTYPE']=np.where((aa['遠交易量']<0) & (aa['短交易量'] < 0)& (aa['LevelArea'] >-200),x_type , aa['TTYPE'])
aa['TJug']=np.where((aa['TTYPE']==x_type) & (aa['過壓力區']>1) , 'T', aa['TJug'])

# 近期已沖量一段時間，  ( 可以找過壓力區>1.1)
#aa[((aa['遠交易量']>0) & (aa['短交易量'] < 0)& (aa['LevelArea'] >-200))].sort_values(by=['過壓力區'], ascending=False)
x_type='TYPE(-+)'
aa['TTYPE']=np.where((aa['遠交易量']>0) & (aa['短交易量'] < 0)& (aa['LevelArea'] >-200) ,x_type , aa['TTYPE'])
aa['TJug']=np.where((aa['TTYPE']==x_type) & (aa['過壓力區'] > 1.1) , 'T', aa['TJug'])

#++ 沒特別發現規則
#bb=aa[((aa['遠交易量']<0) & (aa['短交易量'] > 0)& (aa['LevelArea'] >-200))].sort_values(by=['過壓力區'], ascending=False)
x_type='TYPE(+-)'
aa['TTYPE']=np.where((aa['遠交易量']<0) & (aa['短交易量'] > 0)& (aa['LevelArea'] >-200),x_type , aa['TTYPE'])
aa['TJug']=np.where((aa['TTYPE']==x_type) & (aa['Volume_Price_Change_sum'] > 0) &(aa['MA_break']) , 'T', aa['TJug'])

# 火熱狀態 ( 可以找過壓力區>1.25)
#aa[((aa['遠交易量']>0) & (aa['短交易量'] > 0)& (aa['LevelArea'] >-200))].sort_values(by=['過壓力區'], ascending=False)
x_type='TYPE(++)'
aa['TTYPE']=np.where((aa['遠交易量']>0) & (aa['短交易量'] > 0)& (aa['LevelArea'] >-200) ,x_type , aa['TTYPE'])
aa['TJug']=np.where((aa['TTYPE']==x_type) & (aa['Volume_Price_Change_sum'] > 0)& (aa['過壓力區'] > 1.09) , 'T', aa['TJug'])

In [14]:
## 取得其他資訊
# 读取 Excel 文件
stock_info = pd.read_excel('Result/StockInfo.xlsx', engine='openpyxl')
# 确保两个 DataFrame 的 `stock_number` 列为字符串类型
aa['stock_number'] = aa['stock_number'].astype(str)

stock_info['stock_number'] = stock_info['stock_number'].astype(str)
stock_info['殖利率']=stock_info['殖利率'].str.replace("%", "")
stock_info['總市值'] = (stock_info['總市值'] .str.replace(",", ""))
stock_info['一年內最大量'] = (stock_info['一年內最大量'] .str.replace(",", ""))
stock_info['一年內最低量'] = (stock_info['一年內最低量'] .str.replace(",", ""))
 
# 合并两个 DataFrame
aa = pd.merge(left=aa, right=stock_info, on="stock_number",how='left')


aa['營收比重'] = aa['營收比重'].fillna("-")

In [15]:
def volume_radio(row):
    return str(format(float(row['成交股數'])/float(row['一年內最大量'])*100, '.2f'))    

aa['交易量比']  =aa.apply(volume_radio, axis=1)

#-----------------------------------------------------

In [16]:
aa.columns.values

array(['日期', '成交股數', '成交金額', '開盤價_x', '最高價_x', '最低價_x', '收盤價_x', '漲跌價差',
       '成交筆數', 'Year', 'Date', 'Date_', '年月日', 'EMA_12', 'EMA_26', 'MACD',
       'MACD-SL', 'Signal_Line', 'DIF', 'MACD_golden_cross', 'Lowest_Low',
       'Highest_High', '%K', '%D', 'KD_golden_cross', 'Volume_MA_short',
       'Volume_MA_long', 'Volume_Oscillator', 'Volume_breakout',
       'MA_short', 'MA_long', 'MA_break', 'MA_short_volatility',
       'MA_long_volatility', 'Is_Entangled', 'MA_short_change',
       'MA_long_change', 'Bollinger_MA', 'Bollinger_STD',
       'Bollinger_upper', 'Bollinger_lower', 'Bollinger_breakout', 'RSI',
       'RSI_rebound', 'Price_Change', 'Volume_Price_Change',
       'Volume_Price_Change_sum', 'Volume_Change', 'Volume_Change_sum',
       'Bar_Color', 'Close_above_MA', 'Close_above_MA_5days', 'Analysis',
       'Signal_Balance', 'Signal_Balance_roll',
       'Signal_Balance_roll_diff', 'Signal_Balance_uptrend',
       'Signal_Balance_uptrend_3days', 'Trend', 'Weighted_Sign

In [17]:
aa['_quote'] = pd.to_numeric(aa['_quote'], errors='coerce')
aa['交易量比'] = pd.to_numeric(aa['交易量比'], errors='coerce')
aa['短交易量'] = pd.to_numeric(aa['短交易量'], errors='coerce')
aa['本益比'] = pd.to_numeric(aa['本益比'], errors='coerce')
aa['_quote'] = pd.to_numeric(aa['_quote'], errors='coerce')
_filter= aa[
            (aa['stock_number']!='')
            #& ((aa['%K']>aa['%D']*1.01 ) or(aa['%K']>aa['%D'] &aa['%D']=100 ))
            & ((aa['%K'] >= aa['%D'] * 1.015) | ((aa['%K'] > aa['%D']) & (aa['%D'] == 100)))
       #     &( (aa['MA_short']>aa['MA_long']*1.02 ))
            # & (aa['Volume_Price_Change_sum'] > 1000)
             & (aa['短交易量']>0 )
            ]
#bb=aa
bb=_filter
len(list(set(_filter['stock_number'])))

1088

In [18]:
bb=aa[ 
    (aa['stock_number']!='')
     #&   (aa['日期']=='2025-01-20')
    # & (aa['stock_number'].isin(list(set(_filter['stock_number']))))
    # &((aa['%K'] >= aa['%D'] * 1) | ((aa['%K'] > aa['%D']) & (aa['%D'] == 100)))
    & (aa['Signal_Balance_uptrend_3days']=='True')
    &( (aa['MA_short']>aa['MA_long']*1.01 ))
    & (aa['短交易量']>-10 )
    ].drop_duplicates()

len(list(set(bb['stock_number'])))

88

In [19]:
adate='2025-01-21'
#--------------------------------------------------------
output_col=['stock_number','日期','_quote',
            '收盤價_x' ,'LevelArea','交易量比',
            #'Trend',
            'Trend',
            '短交易量','遠交易量','量增','Volume_Price_Change_sum',
            'now_price',
            'change_quote','diff_quote', 
            'Type0', 'Type1', 'Type2',
            '殖利率','每股淨值(元)','每股營收(元)',
            '本益比',
            '同業平均本益比','總市值','營收比重',
            'Type'
           ]
#--------------------------------------------------------
bb[output_col] = bb[output_col].fillna("0")
bb[output_col] = bb[output_col].astype(str)
bb[output_col] = bb[output_col].replace(["NaN","nan","None", ""], "0")


to_excel='check.xlsx' 
bb[output_col].drop_duplicates().sort_values(by=['stock_number','日期',], ascending=False).to_excel(to_excel, index=False)

to_excel1='check1.xlsx' 
bb[output_col][ (aa['日期']==adate) ].drop_duplicates().sort_values(by=['stock_number','日期',], ascending=False).to_excel(to_excel1, index=False)

to_excel1='check2.xlsx' 
bb[output_col][ (aa['日期']==adate) 
               & (aa['短交易量']>99 )& (aa['遠交易量']>-40 )& (aa['LevelArea']<70 )  
              ].drop_duplicates().sort_values(by=['stock_number','日期',], ascending=False).to_excel(to_excel1, index=False)



cc=aa.copy()

cc[output_col] = cc[output_col].astype(str)
cc[output_col] = cc[output_col].replace(["NaN","nan","None", ""], "0")

to_excel='checkall.xlsx' 
print(len(cc))
cc[output_col][ (aa['日期']==adate)
                #& ((aa['Trend']=='均線:橫盤震盪') | (aa['Trend']=='均線:上漲趨勢') )
                & ( (aa['Trend']=='均線:上漲趨勢') )
              ].drop_duplicates().sort_values(by=['stock_number','日期',], ascending=False).to_excel(to_excel, index=False)
print(len(cc))

myList=['1101', '1215','1599', '2301', '2349', '2367', '2376', '2402', '4564', '4953', '4966', '6117']
to_excel1='check3.xlsx' 
cc[output_col][ (aa['日期']==adate) & (aa['stock_number'].isin(myList))].drop_duplicates().sort_values(by=['stock_number','日期',], ascending=False).to_excel(to_excel1, index=False)


print("近期表現佳")
print("http://localhost/IT/index.html")

print("近期表現佳太陽圖")
print("http://localhost/IT/index2.html")

print("本日ALL")
print("http://localhost/IT/index.html?d=A")

print("本日資金分類流向(all)")
print("http://localhost/IT/index3.html?d=A")

print("本日資金分類流向")
print("http://localhost/IT/index3.html")

print("本日表現佳")
print("http://localhost/IT/index.html?d=T")
print("特別觀察")
print("http://localhost/IT/index.html?d=S")

print("我的")
print("http://localhost/IT/index.html?d=M")

C:\Users\user\AppData\Local\Temp\ipykernel_12932\1568003854.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bb[output_col][ (aa['日期']==adate) ].drop_duplicates().sort_values(by=['stock_number','日期',], ascending=False).to_excel(to_excel1, index=False)
C:\Users\user\AppData\Local\Temp\ipykernel_12932\1568003854.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bb[output_col][ (aa['日期']==adate)


13075
13075
近期表現佳
http://localhost/IT/index.html
近期表現佳太陽圖
http://localhost/IT/index2.html
本日ALL
http://localhost/IT/index.html?d=A
本日資金分類流向(all)
http://localhost/IT/index3.html?d=A
本日資金分類流向
http://localhost/IT/index3.html
本日表現佳
http://localhost/IT/index.html?d=T
特別觀察
http://localhost/IT/index.html?d=S
我的
http://localhost/IT/index.html?d=M


function getMissingElements(array1, array2) {
    return array1.filter(element => !array2.includes(element));

}
array1=app.jsonData.map(x=>x.stock_number)
array2=app.filteredData.map(x=>x.stock_number)
getMissingElements(array1, array2);